In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import VotingClassifier
import numpy as np

# อ่านข้อมูล
df = pd.read_excel('DatasetEng3.xlsx')

# ใช้ LabelEncoder แปลงคอลัมน์ 'Color', 'Smell', 'Quality', และ 'fish'
label_encoders = {}
categorical_columns = ['Color', 'Smell', 'Quality', 'fish']


for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X = df.drop(columns=['Quality'])  # เอาคอลัมน์ 'Quality' ออกจาก X
y1 = df['Quality']
y2 = df['fish']

X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(
    X, y1, y2, test_size=0.2, random_state=42)

# ปรับปรุงตัวแปรเป็นตัวเลข (ถ้าจำเป็น)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



oversampler_y1 = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_resampled_y1, y1_train_resampled = oversampler_y1.fit_resample(X_train, y1_train)

# Oversampling ข้อมูล y2
oversampler_y2 = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_train_resampled_y2, y2_train_resampled = oversampler_y2.fit_resample(X_train, y2_train)

# Ensure the LabelEncoder has seen all possible labels in the 'fish' column
le_fish = LabelEncoder()
le_fish.fit(y2_train_resampled)  # Fit on the oversampled training data



# สร้าง Base Classifiers
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(n_estimators=100, random_state=42))
]


# สร้าง VotingClassifier โดยรวม Base Classifiers
ensemble_classifier = VotingClassifier(estimators=base_classifiers, voting='hard')

# ฝึกโมเดล VotingClassifier บนชุดข้อมูล y1 ที่ถูก Oversampled
ensemble_classifier.fit(X_train_resampled_y1, y1_train_resampled)

# ใช้ ensemble_classifier ทำนายผล y1 บนชุดทดสอบ
y1_pred = ensemble_classifier.predict(X_test)

# ประเมินประสิทธิภาพของโมเดลสำหรับ y1
accuracy_y1 = accuracy_score(y1_test, y1_pred)


# สร้าง VotingClassifier ใหม่โดยรวม Base Classifiers
ensemble_classifier = VotingClassifier(estimators=base_classifiers, voting='hard')

# ฝึกโมเดล VotingClassifier บนชุดข้อมูล y2 ที่ถูก Oversampled
ensemble_classifier.fit(X_train_resampled_y2, y2_train_resampled)

# ใช้ ensemble_classifier ทำนายผล y2 บนชุดทดสอบ
y2_pred = ensemble_classifier.predict(X_test)

# ประเมินประสิทธิภาพของโมเดลสำหรับ y2
accuracy_y2 = accuracy_score(y2_test, y2_pred)


user_input = {
    'DO': float(input("Enter DO value: ")),
    'Color': input("Enter Color value: "),
    'Smell': input("Enter Smell value: "),
    'TDS (ppm)': float(input("Enter TDS (ppm) value: ")),
    'EC (microsecond/cm)': float(input("Enter EC (microsecond/cm) value: ")),
    'PH': float(input("Enter PH value: ")),
    'Area': float(input("Enter Area value: ")),
    'time': float(input("Enter time value: ")),
     # Replace 'default_value' with an appropriate default value
}

user_input_df = pd.DataFrame([user_input])
user_input_df['Color'] = label_encoders['Color'].transform(user_input_df['Color'])
user_input_df['Smell'] = label_encoders['Smell'].transform(user_input_df['Smell'])

# ใช้ X_train.columns เพื่อตรวจสอบลำดับคอลัมน์ที่สอดคล้องกัน


# เพิ่มคอลัมน์ 'fish' จาก DataFrame หลัก (X)
user_input_df['fish'] = X['fish']

# แปลงเป็น NumPy array
user_input_np = user_input_df.to_numpy()

# ปรับขนาดข้อมูลอินพุตของผู้ใช้
user_input_scaled = scaler.transform(user_input_np)

# Make predictions for 'คุณภาพของน้ำ'
quality_prediction = ensemble_classifier.predict(user_input_scaled)

# Make predictions for 'จำนวนปลา'
quantity_prediction = ensemble_classifier.predict(user_input_scaled)

# สำหรับ y1_pred (คุณภาพของน้ำ)
y1_pred_labels = label_encoders['Quality'].inverse_transform(y1_pred)

# สำหรับ y2_pred (จำนวนปลา)
y2_pred_labels = label_encoders['fish'].inverse_transform(y2_pred)

# แสดงผลลัพธ์
print("คุณภาพของน้ำทำนาย:", y1_pred_labels[0])
print("จำนวนปลาทำนาย:", y2_pred_labels[0])


# สร้าง VotingClassifier โดยรวม Base Classifiers
ensemble_classifier = VotingClassifier(estimators=base_classifiers, voting='hard')

# ฝึกโมเดล VotingClassifier บนชุดข้อมูล y1 ที่ถูก Oversampled
ensemble_classifier.fit(X_train_resampled_y1, y1_train_resampled)



# ใช้ ensemble_classifier ทำนายผล y1 บนชุดทดสอบ
y1_pred = ensemble_classifier.predict(X_test)

# ประเมินประสิทธิภาพของโมเดลสำหรับ y1
accuracy_y1 = accuracy_score(y1_test, y1_pred)

# สร้าง VotingClassifier ใหม่โดยรวม Base Classifiers
ensemble_classifier = VotingClassifier(estimators=base_classifiers, voting='hard')

# ฝึกโมเดล VotingClassifier บนชุดข้อมูล y2 ที่ถูก Oversampled
ensemble_classifier.fit(X_train_resampled_y2, y2_train_resampled)

# ใช้ ensemble_classifier ทำนายผล y2 บนชุดทดสอบ
y2_pred = ensemble_classifier.predict(X_test)
y1_pred_labels = label_encoders['Quality'].inverse_transform(y1_pred)

# Transform y2_pred using the LabelEncoder
y2_pred_labels = le_fish.inverse_transform(y2_pred)

# ประเมินประสิทธิภาพของโมเดลสำหรับ y2
accuracy_y2 = accuracy_score(y2_test, y2_pred)


# แสดงผลลัพธ์

print("ความแม่นยำของการทำนายคุณภาพของน้ำ:", accuracy_y1)

print("ความแม่นยำของการทำนายจำนวนปลา:", accuracy_y2)

Enter DO value: 6.5
Enter Color value: brown
Enter Smell value: yes
Enter TDS (ppm) value: 625
Enter EC (microsecond/cm) value: 1230
Enter PH value: 6.23
Enter Area value: 19
Enter time value: 17.55


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


คุณภาพของน้ำทำนาย: Best
จำนวนปลาทำนาย: Many
ความแม่นยำของการทำนายคุณภาพของน้ำ: 1.0
ความแม่นยำของการทำนายจำนวนปลา: 1.0
